In [66]:
from dataclasses import dataclass, field
from enum import auto, Enum
from math import inf
from queue import PriorityQueue

In [84]:
class Move(Enum):
    FORWARD = auto()
    CLOCKWISE = auto()
    COUNTERCLOCKWISE = auto()

    def get_value(self):
        match self:
            case self.FORWARD:
                value = 1
            case self.CLOCKWISE | self.COUNTERCLOCKWISE:
                value = 1000

        return value

class Direction(Enum):
    UP = (0, -1)
    RIGHT = (1, 0)
    DOWN = (0, 1)
    LEFT = (-1, 0)

    def turn(self, move: Move):
        directions = [Direction.UP, Direction.RIGHT, Direction.DOWN, Direction.LEFT]
        idx = directions.index(self)  # Ensure 'self' matches an Enum in 'directions'
    
        if move == Move.CLOCKWISE:
            return directions[(idx + 1) % 4]
        elif move == Move.COUNTERCLOCKWISE:
            return directions[(idx - 1) % 4]
        return self

    

@dataclass(slots=True, frozen=True)
class MoveTile:
    x: int
    y: int

def find_shortest_path(self):
    start = self.position
    target = self.objective

    # Priority queue for Dijkstra's algorithm
    pq = PriorityQueue()
    pq.put((0, start, self.current_direction))  # (cost, position, direction)

    # Distance dictionary
    distances = {(start, self.current_direction): 0}
    visited = set()

    while not pq.empty():
        current_cost, current_pos, current_direction = pq.get()

        if (current_pos, current_direction) in visited:
            continue
        visited.add((current_pos, current_direction))

        if current_pos == target:
            return current_cost  # Shortest path cost

        # Evaluate all possible moves
        for move in Move:
            if move == Move.FORWARD:
                delta = current_direction.value
                next_pos = (current_pos[0] + delta[0], current_pos[1] + delta[1])

                if (
                    next_pos in visited
                    or next_pos in self.obstacles
                    or not self._pos_in_bounds(next_pos)
                ):
                    continue

                move_cost = move.get_value()
                new_cost = current_cost + move_cost

                if new_cost < distances.get((next_pos, current_direction), inf):
                    distances[(next_pos, current_direction)] = new_cost
                    pq.put((new_cost, next_pos, current_direction))

            elif move in {Move.CLOCKWISE, Move.COUNTERCLOCKWISE}:
                new_direction = current_direction.turn(move)
                move_cost = move.get_value()
                new_cost = current_cost + move_cost

                if new_cost < distances.get((current_pos, new_direction), inf):
                    distances[(current_pos, new_direction)] = new_cost
                    pq.put((new_cost, current_pos, new_direction))

    return inf

In [85]:
with open("data/test.txt", "r") as file:
    puzzle = [line.strip() for line in file.readlines()]
    
maze = ReindeerMaze().setup(puzzle)
maze.find_shortest_path(), puzzle

ValueError: <Direction.UP: (0, -1)> is not in list